<a href="https://colab.research.google.com/github/MShiloni22/DDBMS_Project_A/blob/master/DDBMS_Project_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 38.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=eba20be9ee0a211c452e50d0260d68f38d0d3c598e362ce9b1b4c623f2b060a1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import findspark
import datetime as dt

findspark.init()
from pyspark.sql import SparkSession
def init_spark(app_name: str):
 spark = SparkSession.builder.appName(app_name).getOrCreate()
 sc = spark.sparkContext
 return spark, sc
spark, sc = init_spark('demo')
sc

# Section 1 - Data Extraction
# Current file = queries.csv

# Transform the file to tsv format
import csv

with open('/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/queries.csv','r') as csvin,\
    open('/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/queries.tsv', 'w') as tsvout:
    csvin = csv.reader(csvin)
    tsvout = csv.writer(tsvout, delimiter='\t')
    for row in csvin:
        tsvout.writerow(row)

from pyspark.sql import SparkSession,Row, Column
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
queries_file = '/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/queries.tsv'
df = spark.read.csv(queries_file, header='True', inferSchema='True', sep='\t')


column_names = ["genres", "lang", "actors", "director", "cities", "country", 
                "from_realese_date", "production_company"]
# For all the above columns
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # Removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "'[]", "")\
                .alias(temp_name_1))\
  .drop(name)

  # Converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

print("Queries table:")
df.show()
queries_df = df

# ------------------------------------------------------------------------------

# Current file: credits.csv
import re
credits_file = '/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/credits.csv'
df = spark.read.csv(credits_file, header='True', inferSchema='True')

# load the data as you did before,
# just now change the delimiter to get evreything together
credits = spark.read.format("csv")\
.option("delimiter", "\t")\
.option("header","true")\
.option("inferSchema", "true")\
.load("/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/credits.csv")
prog = re.compile('\\[(.*?)\\]')
second_match = F.udf(lambda x: prog.findall(x)[1])
id_extract = F.udf(lambda x: x.split(",")[-1])
credits = credits\
.withColumn("id", id_extract("cast,crew,id"))\
.withColumn("cast", F.regexp_extract(F.col("cast,crew,id"), '\\[(.*?)\\]', 0
))\
.withColumn("crew", F.concat(F.lit("["),second_match("cast,crew,id"), F.lit(
"]")))\
.select("cast", "crew", "id")
df = credits
# df.printSchema()
column_names = ["cast", "crew"]
# For all the above columns
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # Removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "\\{\\[\\]'\\}", "")\
                .alias(temp_name_1))\
  .drop(name)

  # Converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

# For cast column - udf for extracting actors' names only from cast json string
actors_udf = F.udf(lambda arr: [arr[i][7:] for i in range(len(arr)) if i % 8 == 5])
df = df.withColumn('actors', actors_udf(F.col("cast")))\
  .drop("cast")

# For crew column - udf for extracting directors' names only from crew json string
directors_udf = F.udf(lambda arr: [arr[i+1][7:] for i in range(len(arr))
 if arr[i] == " job: Director"])
df = df.withColumn('directors', directors_udf(F.col("crew")))\
  .drop("crew")

# Converting arrays strings to arrays of strings
column_names = ["actors", "directors"]
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # Removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "\\{\\[\\]'\\}", "")\
                .alias(temp_name_1))\
  .drop(name)

  # Converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

print("Credits table:")
df.show()
credits_df = df

# ------------------------------------------------------------------------------

# Current file: movies.csv
movies_file = '/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/movies.csv'
df = spark.read.csv(movies_file, header='True', inferSchema='True')

# Doing the same process for all columns
column_names = ["genres", "production_companies", "production_countries", 
                "spoken_languages", "cities"]
for name in column_names:
  temp_name_1 = name + "1"
  temp_name_2 = name + "2"
  # Removing irrelevant chars
  df = df.select("*", F.translate(F.col(name), "\\{\\[\\]'\\}", "")\
                .alias(temp_name_1))\
  .drop(name)

  # Converting arrays strings to arrays of strings
  df = df.select("*", F.split(F.col(temp_name_1),",").alias(temp_name_2)) \
      .drop(temp_name_1)
  df = df.withColumnRenamed(temp_name_2,name)

# Finished working on cities column, and seperating production_companies
# because it has different structure
column_names = ["genres", "production_countries", "spoken_languages"]

# For each column - udf for extracting names only from json string
name_udf = F.udf(lambda arr: [arr[i][7:] for i in range(len(arr)) if i % 2 == 1])
for c in column_names:
  c_1 = c + "1"
  df = df.withColumn(c_1, name_udf(F.col(c)))\
    .drop(c)
prod_udf = F.udf(lambda arr: [arr[i][6:] for i in range(len(arr)) if i % 2 == 0])
df = df.withColumn("production_companies1", prod_udf(F.col("production_companies")))\
  .drop("production_companies")

# Renameing columns names
column_names = ["genres", "production_companies", "production_countries", 
                "spoken_languages"]
for name in column_names:
  current_name = name + "1"
  df = df.withColumnRenamed(current_name,name)

print("Movies table:")
df.show()
movies_df = df

Mounted at /content/drive
Queries table:
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|user_id|              genres|                lang|              actors|            director|              cities|             country|from_realese_date|  production_company|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|    981| [Western,  Mystery]|  [English,  Srpski]|                  []|      [Nae Caranfil]|  [Haifa,  Tiberias]|                  []|           [2012]|[Katakuri-ke no K...|
|   3775|  [Action,  Western]|           [English]|                  []|                  []|          [Tel Aviv]|                  []|           [2013]|[Clavius Base,  T...|
|   4095|             [Crime]|[English,  עִבְרִית]|[Kenneth Alton,  ...|            

In [36]:
# Section 4 - Deployment

#
#
#
# Due to many debugging issues that we couldn't overcome, we were unfortunately 
# enforced to make the partition on the original given data.
# There was not any specific damand to implement the fragmentation on out transformed 
# data from part 1.
# In fact, there is no significant difference in this, compared to the implementation
# on the data of Part 1, since the considerations will in any case be the same 
# considerations, which we detailed in the document of Part 3.
#
#

# creating copies for all sites and all tables needed
users_file = '/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/users.csv'

tel_aviv_users = spark.read.csv(users_file, header='True', inferSchema='True')
jerusalem_users = spark.read.csv(users_file, header='True', inferSchema='True')
haifa_users = spark.read.csv(users_file, header='True', inferSchema='True')
gesher_users = spark.read.csv(users_file, header='True', inferSchema='True')
eilat_users = spark.read.csv(users_file, header='True', inferSchema='True')

tickets_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/tickets.csv',
                            header='True', inferSchema='True')

tel_aviv_tickets = tickets_df.alias('tel_aviv_tickets')
jerusalem_tickets = tickets_df.alias('jerusalem_tickets')
haifa_tickets = tickets_df.alias('haifa_tickets')
gesher_tickets = tickets_df.alias('gesher_tickets')
eilat_tickets = tickets_df.alias('eilat_tickets')

# applying our desgin on the tables
# deploymenting users
jerusalem_users = jerusalem_users.where(jerusalem_users.user_location == "Jerusalem")
tel_aviv_users = tel_aviv_users.where(tel_aviv_users.user_location == "Tel Aviv")
haifa_users = haifa_users.where(haifa_users.user_location == "Haifa")
gesher_users = gesher_users.where(gesher_users.user_location == "Kibuts Gesher")
eilat_users = eilat_users.where(eilat_users.user_location == "Eilat")

# deploymenting tickets
jerusalem_tickets = jerusalem_tickets.where(jerusalem_tickets.city == "Jerusalem")
tel_aviv_tickets = tel_aviv_tickets.where(tel_aviv_tickets.city == "Tel Aviv")
haifa_tickets = haifa_tickets.where(haifa_tickets.city == "Haifa")
gesher_tickets = gesher_tickets.where(gesher_tickets.city == "Tiberias")
eilat_tickets = eilat_tickets.where(eilat_tickets.city == "Eilat")

# deploymenting movies
movies_file = '/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/movies.csv'
df = spark.read.csv(movies_file, header='True', inferSchema='True')

df_2 = df.select(F.col("movie_id"), F.col("genres"), F.col("overview"),
                 F.col("production_companies"), F.col("production_countries"),
                 F.year(F.to_date(F.col("release_date"), "dd/MM/yyyy")).cast("int").alias("release_date"),
                 F.col("revenue"), F.col("spoken_languages"), F.col("tagline"),
                 F.col("title"), F.col("cities"))

df_3 = df.select(F.col("movie_id"), F.col("genres"),
                 F.col("production_companies"), F.col("production_countries"),
                 F.year(F.to_date(F.col("release_date"), "dd/MM/yyyy")).cast("int").alias("release_date"),
                 F.col("spoken_languages"), F.col("cities"))


jerusalem_movies = df_2.where((F.col("production_countries").like("%Israel%"))
                            & (F.col("release_date") >= 1990))
tel_aviv_movies = df_3.where(((F.col("genres").like("%Action%") | F.col("genres").like("%Drama%")
 | F.col("genres").like("%Science Fiction%")) | F.col("production_countries").like("%Israel%"))
  & (F.col("release_date") >= 1990))
haifa_movies = df_3.where((F.col("genres").like("%Drama%") | ~F.col("spoken_languages").like("%עִבְרִית%"))
                          & (F.col("release_date") >= 1990))
gesher_movies = df_3.where((F.col("genres").like("%Drama%") | F.col("genres").like("%Documentary%") 
  | F.col("genres").like("%Family%") | ~F.col("spoken_languages").like("%עִבְרִית%"))
  & (F.col("release_date") >= 1990))
eilat_movies = df_3.where((~F.col("spoken_languages").like("%עִבְרִית%")) 
  | (F.col("production_companies").like("%Pixar Animation Studios%"))
  | (F.col("production_companies").like("%Walt Disney Pictures%")) 
  | (F.col("production_companies").like("%Warner Bros%")))

print("Jerusalem Movies:")
jerusalem_movies.show()
print("Tel Aviv Movies:")
tel_aviv_movies.show()
print("Haifa Movies:")
haifa_movies.show()
print("Gesher Movies:")
gesher_movies.show()
print("Eilat Movies:")
eilat_movies.show()

# deploymenting credits
credits_file = '/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/credits.csv'
df = spark.read.csv(credits_file, header='True', inferSchema='True')

jerusalem_credits = df.select("*")
tel_aviv_credits = df.where(F.col("cast").like("%Tom Hanks%") 
  | F.col("cast").like("%Brad Pitt%") 
  | F.col("cast").like("%Johnny Depp%"))

print("Jerusalem Credits:")
jerusalem_credits.show()
print("Tel Aviv Credits:")
tel_aviv_credits.show()
print("Other sites shouldn't contain this table acoording to our design")
print()

# deploymenting queries
queries_file = '/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/queries.csv'
df = spark.read.csv(queries_file, header='True', inferSchema='True')
eilat_queries = df.select("*")


# Final fragmantation
final_fragmantation = {"Jerusalem": [jerusalem_users, jerusalem_tickets, jerusalem_movies,
                                     jerusalem_credits],
                       "Tel Aviv": [tel_aviv_users, tel_aviv_tickets, tel_aviv_movies,
                                    tel_aviv_credits],
                       "Haifa": [haifa_users, haifa_tickets, haifa_movies],
                       "Kibuts Gesher": [gesher_users, gesher_tickets, gesher_movies],
                       "Eilat": [eilat_users, eilat_tickets, eilat_movies, eilat_queries]}

print("Our final fragmantation:")
print(final_fragmantation)

for site in final_fragmantation:
  for i, dataframe in enumerate(final_fragmantation[site]):
    tname = [k for k, v in locals().items() if v == dataframe]
    tname = tname[0] if tname[0] != "dataframe" else tname[1]
    dataframe.write.option("header", True).csv(f"/content/drive/MyDrive/Colab Notebooks/DDBMS/DDBMS_proj_A_files/final_freg/{site}/{tname}")

Jerusalem Movies:
+--------+--------------------+--------------------+--------------------+--------------------+------------+-------+--------------------+--------------------+--------------------+--------------------+
|movie_id|              genres|            overview|production_companies|production_countries|release_date|revenue|    spoken_languages|             tagline|               title|              cities|
+--------+--------------------+--------------------+--------------------+--------------------+------------+-------+--------------------+--------------------+--------------------+--------------------+
|    8056|[{'id': 18, 'name...|In 1941, the inha...|[{'name': 'Canal ...|[{'iso_3166_1': '...|        1998|      0|[{'iso_639_1': 'f...|                null|       Train of Life|['Eilat', 'Jerusa...|
|   28308|[{'id': 18, 'name...|An ancient skelet...|[{'name': 'Helkon...|[{'iso_3166_1': '...|        2001|      0|[{'iso_639_1': 'a...|It would change h...|            The Body|['Ha